# Drzewa Decyzyjne

In [21]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

## Import zbioru Secondary Mushroom

In [22]:
mushroom_species_info_df = pd.read_csv("../dbs/MushroomDataset/primary_data.csv", sep=";")
mushroom_species_info_df.head()


,family,name,class,cap-diameter,cap-shape,Cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,Spore-print-color,habitat,season
0,Amanita Family,Fly Agaric,p,"[10, 20]","[x, f]","[g, h]","[e, o]",[f],[e],NaN,...,[s],[y],[w],[u],[w],[t],"[g, p]",NaN,[d],"[u, a, w]"
1,Amanita Family,Panther Cap,p,"[5, 10]","[p, x]",[g],[n],[f],[e],NaN,...,NaN,[y],[w],[u],[w],[t],[p],NaN,[d],"[u, a]"
2,Amanita Family,False Panther Cap,p,"[10, 15]","[x, f]",NaN,"[g, n]",[f],[e],NaN,...,NaN,NaN,[w],[u],[w],[t],"[e, g]",NaN,[d],"[u, a]"
3,Amanita Family,The Blusher,e,"[5, 15]","[x, f]",NaN,[n],[t],NaN,NaN,...,[b],NaN,[w],[u],[w],[t],[g],NaN,[d],"[u, a]"
4,Amanita Family,Death Cap,p,"[5, 12]","[x, f]",[h],[r],[f],NaN,[c],...,NaN,NaN,[w],[u],[w],[t],"[g, p]",NaN,[d],"[u, a]"


In [23]:
df = pd.read_csv("../dbs/MushroomDataset/secondary_data.csv", sep=";")
df.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


## EDA
Wykorzytane AutoEDA z `ydata-profiling`.

In [24]:
# AutoEDA
profile = ProfileReport(df, title="Secondary Mushroom Data Report", explorative=True)
profile.to_file("mushroom_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/miniconda3/envs/py310/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'o'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# sns.pairplot(df, hue='class', palette='coolwarm')
# plt.show()

In [25]:
# profile.to_widgets()

## Informacje o zbiorze danych ze strony

### Classification
- **Edibility**: Binary class divided into edible (`e`) and poisonous (`p`). Poisonous category also includes mushrooms of unknown edibility.

### Features
1. **Cap Diameter** (Metric)
   - Type: Float number (cm)

2. **Cap Shape** (Nominal)
   - Options: Bell (`b`), Conical (`c`), Convex (`x`), Flat (`f`), Sunken (`s`), Spherical (`p`), Others (`o`)

3. **Cap Surface** (Nominal)
   - Options: Fibrous (`i`), Grooves (`g`), Scaly (`y`), Smooth (`s`), Shiny (`h`), Leathery (`l`), Silky (`k`), Sticky (`t`), Wrinkled (`w`), Fleshy (`e`)

4. **Cap Color** (Nominal)
   - Options: Brown (`n`), Buff (`b`), Gray (`g`), Green (`r`), Pink (`p`), Purple (`u`), Red (`e`), White (`w`), Yellow (`y`), Blue (`l`), Orange (`o`), Black (`k`)

5. **Does Bruise/Bleed** (Nominal)
   - Options: Bruises or Bleeding (`t`), No (`f`)

6. **Gill Attachment** (Nominal)
   - Options: Adnate (`a`), Adnexed (`x`), Decurrent (`d`), Free (`e`), Sinuate (`s`), Pores (`p`), None (`f`), Unknown (`?`)

7. **Gill Spacing** (Nominal)
   - Options: Close (`c`), Distant (`d`), None (`f`)

8. **Gill Color** (Nominal)
   - Options: See Cap Color + None (`f`)

9. **Stem Height** (Metric)
   - Type: Float number (cm)

10. **Stem Width** (Metric)
    - Type: Float number (mm)

11. **Stem Root** (Nominal)
    - Options: Bulbous (`b`), Swollen (`s`), Club (`c`), Cup (`u`), Equal (`e`), Rhizomorphs (`z`), Rooted (`r`)

12. **Stem Surface** (Nominal)
    - Options: See Cap Surface + None (`f`)

13. **Stem Color** (Nominal)
    - Options: See Cap Color + None (`f`)

14. **Veil Type** (Nominal)
    - Options: Partial (`p`), Universal (`u`)

15. **Veil Color** (Nominal)
    - Options: See Cap Color + None (`f`)

16. **Has Ring** (Nominal)
    - Options: Ring (`t`), None (`f`)

17. **Ring Type** (Nominal)
    - Options: Cobwebby (`c`), Evanescent (`e`), Flaring (`r`), Grooved (`g`), Large (`l`), Pendant (`p`), Sheathing (`s`), Zone (`z`), Scaly (`y`), Movable (`m`), None (`f`), Unknown (`?`)

18. **Spore Print Color** (Nominal)
    - Options: See Cap Color

19. **Habitat** (Nominal)
    - Options: Grasses (`g`), Leaves (`l`), Meadows (`m`), Paths (`p`), Heaths (`h`), Urban (`u`), Waste (`w`), Woods (`d`)

20. **Season** (Nominal)
    - Options: Spring (`s`), Summer (`u`), Autumn (`a`), Winter (`w`)


## Wnioski EDA:

### Brakujące wartości:
Powyżej 50%:
- `veil-type`: 94.8% brakujących wartości, reszta oznaczona jako `u` - zamiana kolumny na binarną.
- `spore-print-color`: 89.6% brakujących wartości, reszta: 7 kategorii - wpisanie `none` jako brakująca wartość.
- `veil-color`: 87.9% brakujących wartości, reszta: 5 kategorii - wpisanie `f` jako brakująca wartość (tak jak w opisie wyżej).
- `stem-root`: 84.4% brakujących wartości, reszta: 5 kategorii - wpisanie `none` jako brakująca wartość.
- `stem-surface`: 62.4% brakujących wartości, reszta: 7 kategorii - wpisanie `f` jako brakująca wartość (tak jak w opisie wyżej).

Poniżej 50%:
- `gill-spacing`: 41% brakuje, reszta: 7 kategorii - wpisanie `f` jako brakująca wartość.
- `cap-surface`: 23.1% brakuje, reszta: 11 kategorii - wpisanie `unknown` jako brakująca wartość.
- `gill-attachment`: 16.2% brakuje, reszta: 7 kategorii - wpisanie `none` jako brakująca wartość.
- `ring-type`: 4% brakuje, reszta 8 kategorii - wpisanie `unknown` jako brakująca wartość

Numeryczne: 
- `stem-width`: 1.7% wartości zerowych - usunięcie brakujących wierszy
- `stem-height`: 1.7% wartości zerowych - usunięcie brakujących wierszy



## Przygotowanie danych do modelowania


In [27]:
# Convert 'veil-type' to a binary column (e.g., 0 if NaN, 1 if 'u')
df['veil-type'] = np.where(df['veil-type'].isnull(), 0, 1)

# Fill missing values for 'spore-print-color' and 'stem-root' with 'none'
df['spore-print-color'] = df['spore-print-color'].fillna('none')
df['stem-root'] = df['stem-root'].fillna('none')

# Fill missing values for 'veil-color' and 'stem-surface' with 'f'
df['veil-color'] = df['veil-color'].fillna('f')
df['stem-surface'] = df['stem-surface'].fillna('f')

# Fill missing values for 'gill-spacing' with 'f'
df['gill-spacing'] = df['gill-spacing'].fillna('f')

# Fill missing values for 'cap-surface' with 'unknown'
df['cap-surface'] = df['cap-surface'].fillna('unknown')

# Fill missing values for 'gill-attachment' with 'none'
df['gill-attachment'] = df['gill-attachment'].fillna('none')

# Fill missing values for 'ring-type' with 'unknown'
df['ring-type'] = df['ring-type'].fillna('unknown')

# Remove rows with zero values in 'stem-width' and 'stem-height'
df = df[df['stem-width'] != 0]
df = df[df['stem-height'] != 0]


In [28]:
report_after_correction = ProfileReport(df, title="Secondary Mushroom Data Report After Correction", explorative=True)
report_after_correction.to_file("report_after_correction.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/miniconda3/envs/py310/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'o'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Wytrenowanie drzew decyzyjnych


### Przeszukiwanie hiperparametrów

### Wizualizacja drzewa